# Basics `pandas`

:::{note}
If you have not yet set up Python on your computer, you can execute this tutorial in your browser via [Google Colab](https://colab.research.google.com/). Click on the rocket in the top right corner and launch "Colab". If that doesn't work download the `.ipynb` file and import it in [Google Colab](https://colab.research.google.com/).
Then install `pandas` and `numpy` by executing the following command in a Jupyter cell at the top of the notebook.

```sh
!pip install -q pandas numpy
```
:::

[Pandas](http://pandas.pydata.org/) is a an open source library providing tabular data structures and data analysis tools. In other words, if you can imagine the data in an Excel spreadsheet, then Pandas is the tool for the job.

<img src="https://media.geeksforgeeks.org/wp-content/cdn-uploads/creating_dataframe1.png" width="720px" />

:::{note}
Documentation for this package is available at https://pandas.pydata.org/docs/.
:::

## Package Imports

This will be our first experience with _importing_ a package.

Usually we import `pandas` with the _alias_ `pd`.

We might also need `numpy`, Python's main library for numerical computations.

In [2]:
import pandas as pd
import numpy as np
import plotly.io as pio
import plotly.offline as py
pd.options.plotting.backend = "plotly"

## Series

A Series represents a one-dimensional array of data. It is similar to a dictionary consisting of an **index** and **values**, but has more functions. 

:::{note}
Example data on Germany's final six nuclear power plants is from [Wikipedia](https://en.wikipedia.org/wiki/List_of_power_stations_in_Germany#Nuclear).
:::

In [3]:
dictionary = {
    "Neckarwestheim": 1269,
    "Isar 2": 1365,
    "Emsland": 1290,
}
s = pd.Series(dictionary)
s

Neckarwestheim    1269
Isar 2            1365
Emsland           1290
dtype: int64

Arithmetic operations can be applied to the whole `pd.Series`.

In [4]:
s**0.5

Neckarwestheim    35.623026
Isar 2            36.945906
Emsland           35.916570
dtype: float64

We can access the underlying index object if we need to:

In [5]:
s.index

Index(['Neckarwestheim', 'Isar 2', 'Emsland'], dtype='object')

We can get values back out using the index via the `.loc` attribute

In [6]:
s.loc["Isar 2"]

np.int64(1365)

We can pass a list or array to loc to get multiple rows back:

In [7]:
s.loc[["Neckarwestheim", "Emsland"]]

Neckarwestheim    1269
Emsland           1290
dtype: int64

## DataFrame

Series are limited to a single **column**. A more useful Pandas data structure is the **DataFrame**. A DataFrame is basically a bunch of series that share the same index.

In [8]:
data = {
    "capacity": [1269, 1365, 1290],  # MW
    "type": ["PWR", "PWR", "PWR"],
    "start_year": [1989, 1988, 1988],
    "end_year": [np.nan, np.nan, np.nan],
}
df = pd.DataFrame(data, index=["Neckarwestheim", "Isar 2", "Emsland"])
df

,capacity,type,start_year,end_year
Neckarwestheim,1269,PWR,1989,NaN
Isar 2,1365,PWR,1988,NaN
Emsland,1290,PWR,1988,NaN


A wide range of statistical functions are available on both Series and DataFrames.

In [9]:
df.min()

capacity      1269
type           PWR
start_year    1988
end_year       NaN
dtype: object

We can get a single column as a Series using python's getitem syntax on the DataFrame object.

In [10]:
df["capacity"]

Neckarwestheim    1269
Isar 2            1365
Emsland           1290
Name: capacity, dtype: int64

Indexing works very similar to series

In [11]:
df.loc["Emsland"]

capacity      1290
type           PWR
start_year    1988
end_year       NaN
Name: Emsland, dtype: object

But we can also specify the column(s) and row(s) we want to access

In [12]:
df.at["Emsland", "start_year"]

np.int64(1988)

We can also add new columns to the DataFrame:

In [13]:
df["reduced_capacity"] = df.capacity * 0.8
df

,capacity,type,start_year,end_year,reduced_capacity
Neckarwestheim,1269,PWR,1989,NaN,1015.2
Isar 2,1365,PWR,1988,NaN,1092.0
Emsland,1290,PWR,1988,NaN,1032.0


## Sorting Data

We can also sort the entries in dataframes, e.g. alphabetically by index or numerically by column values

In [14]:
df.sort_index()

,capacity,type,start_year,end_year,reduced_capacity
Emsland,1290,PWR,1988,NaN,1032.0
Isar 2,1365,PWR,1988,NaN,1092.0
Neckarwestheim,1269,PWR,1989,NaN,1015.2


In [15]:
df.sort_values(by="capacity", ascending=True)

,capacity,type,start_year,end_year,reduced_capacity
Neckarwestheim,1269,PWR,1989,NaN,1015.2
Emsland,1290,PWR,1988,NaN,1032.0
Isar 2,1365,PWR,1988,NaN,1092.0


## Filtering Data

We can also filter a DataFrame using a boolean series obtained from a condition. This is very useful to build subsets of the DataFrame.

In [16]:
df.capacity > 1300

Neckarwestheim    False
Isar 2             True
Emsland           False
Name: capacity, dtype: bool

In [17]:
df[df.capacity > 1300]

,capacity,type,start_year,end_year,reduced_capacity
Isar 2,1365,PWR,1988,NaN,1092.0


## Modifying Values

In many cases, we want to modify values in a dataframe based on some rule. To modify values, we need to use `.loc` or `.iloc`

In [18]:
df.loc["Isar 2", "capacity"] = 1366
df

,capacity,type,start_year,end_year,reduced_capacity
Neckarwestheim,1269,PWR,1989,NaN,1015.2
Isar 2,1366,PWR,1988,NaN,1092.0
Emsland,1290,PWR,1988,NaN,1032.0


## Time Series

Time indexes are great when handling time-dependent data.

Let's first read some time series data, using the `pd.read_csv()` function, which takes a local file path or a link to an online resource.

The example data hourly time series for Germany in 2015 for:
    
1. electricity demand from [OPSD](https://open-power-system-data.org/) in GW
2. onshore wind capacity factors from [renewables.ninja](https://www.renewables.ninja/) in per-unit of installed capacity
3. offshore wind capacity factors from [renewables.ninja](https://www.renewables.ninja/) in per-unit of installed capacity
4. solar PV capacity factors from [renewables.ninja](https://www.renewables.ninja/) in per-unit of installed capacity
5. electricity day-ahead spot market prices in €/MWh from EPEX Spot zone DE/AT/LU retrieved via [SMARD platform](https://www.smard.de/home)

In [19]:
url = (
    "https://tubcloud.tu-berlin.de/s/pKttFadrbTKSJKF/download/time-series-lecture-2.csv"
)
ts = pd.read_csv(url, index_col=0, parse_dates=True)
ts.index[:5]

DatetimeIndex(['2015-01-01 00:00:00', '2015-01-01 01:00:00',
               '2015-01-01 02:00:00', '2015-01-01 03:00:00',
               '2015-01-01 04:00:00'],
              dtype='datetime64[ns]', freq=None)

We can use Python's _slicing_ notation inside `.loc` to select a date range, and then use the built-in plotting feature of Pandas:

In [20]:
ts.loc["2015-01-01":"2015-03-01", "onwind"].plot()

A common operation is to change the resolution of a dataset by resampling in time, which Pandas exposes through the [resample](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#resampling) function.

:::{note}
The resample periods are specified using pandas [offset index](http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases) syntax.
:::

In [21]:
ts["onwind"].resample("D").mean().plot()

## Groupby Functionality

`DataFrame` objects have a `groupby` method. The simplest way to think about it is that you pass another series, whose values are used to split the original object into different groups.

Here's an example which retrieves the total generation capacity per country:

In [22]:
fn = "https://raw.githubusercontent.com/PyPSA/powerplantmatching/master/powerplants.csv"

In [23]:
df = pd.read_csv(fn, index_col=0)
df.iloc[:5, :10]

,Name,Fueltype,Technology,Set,Country,Capacity,Efficiency,DateIn,DateRetrofit,DateOut
id,,,,,,,,,,
0,Kernkraftwerk Emsland,Nuclear,Steam Turbine,PP,Germany,1336.0,0.33,1988.0,1988.0,2023.0
1,Brokdorf,Nuclear,Steam Turbine,PP,Germany,1410.0,0.33,1986.0,1986.0,2021.0
2,Borssele,Hard Coal,Steam Turbine,PP,Netherlands,485.0,NaN,1973.0,NaN,2034.0
3,Gemeinschaftskernkraftwerk Neckarwestheim,Nuclear,Steam Turbine,PP,Germany,1310.0,0.33,1976.0,1989.0,2023.0
4,Isar,Nuclear,Steam Turbine,PP,Germany,1410.0,0.33,1979.0,1988.0,2023.0


In [24]:
grouped = df.groupby("Country").Capacity.sum()
grouped.head()

Country
Albania                    2370.400000
Austria                   24643.200368
Belgium                   21443.151009
Bosnia and Herzegovina     4827.195964
Bulgaria                  15699.186363
Name: Capacity, dtype: float64

Let's break apart this operation a bit. The workflow with `groupby` can be divided into three general steps:

1. **Split**: Partition the data into different groups based on some criterion.
2. **Apply**: Do some caclulation within each group, e.g. minimum, maximum, sums.
3. **Combine**: Put the results back together into a single object.

<img src="https://miro.medium.com/max/1840/1*JbF6nhrQsn4f-TaSF6IR9g.png" width="720px" />

Grouping is not only possible on a single columns, but also on multiple columns. For instance,
we might want to group the capacities by country **and** fuel type. To achieve this, we pass a list of functions to the `groupby` functions.

In [25]:
capacities = df.groupby(["Country", "Fueltype"]).Capacity.sum().unstack()
capacities

Fueltype,Biogas,Geothermal,Hard Coal,Hydro,Lignite,Natural Gas,Nuclear,Oil,Other,Solar,Solid Biomass,Waste,Wind
Country,,,,,,,,,,,,,
Albania,NaN,NaN,NaN,1743.900000,NaN,NaN,NaN,NaN,98.00000,294.500000,NaN,NaN,234.0000
Austria,NaN,NaN,1331.400000,14205.500368,522.000000,4952.000000,NaN,NaN,NaN,388.300000,NaN,NaN,3244.0000
Belgium,NaN,NaN,3316.000000,1379.751009,NaN,5251.300000,5931.0,131.60000,NaN,484.100000,498.000,259.700,4191.7000
Bosnia and Herzegovina,NaN,NaN,NaN,2213.500000,1911.595964,NaN,NaN,NaN,NaN,267.100000,NaN,NaN,435.0000
Bulgaria,NaN,NaN,1909.191813,2899.540000,4807.354550,990.000000,2080.0,NaN,NaN,2405.100000,NaN,NaN,608.0000
Croatia,NaN,NaN,308.848880,2129.772189,NaN,1440.800000,NaN,NaN,NaN,96.300000,NaN,NaN,1140.0000
Czechia,NaN,NaN,1225.227500,1888.219836,7297.666121,1659.000000,4164.0,NaN,NaN,1293.600000,30.000,NaN,140.0000
Denmark,NaN,NaN,5658.885609,NaN,NaN,1813.200000,NaN,664.00000,70.00000,2256.700000,2542.000,NaN,5670.8000
Estonia,NaN,NaN,NaN,NaN,NaN,389.000000,NaN,1841.00000,NaN,446.200000,NaN,NaN,713.4000


## Exercises

**Task 1:** Provide a list of unique fuel types included in the power plants dataset.

In [26]:
df.Fueltype.unique()

array(['Nuclear', 'Hard Coal', 'Hydro', 'Lignite', 'Natural Gas', 'Oil',
       'Solid Biomass', 'Wind', 'Solar', 'Other', 'Biogas', 'Waste',
       'Geothermal'], dtype=object)

**Task 2:** Filter the dataset by power plants with the fuel type "Hard Coal". How many hard coal power plants are there?

In [27]:
coal = df.loc[df.Fueltype == "Hard Coal"]
coal

,Name,Fueltype,Technology,Set,Country,Capacity,Efficiency,DateIn,DateRetrofit,DateOut,lat,lon,Duration,Volume_Mm3,DamHeight_m,StorageCapacity_MWh,EIC,projectID
id,,,,,,,,,,,,,,,,,,
2,Borssele,Hard Coal,Steam Turbine,PP,Netherlands,485.000000,NaN,1973.0,NaN,2034.0,51.433200,3.716000,NaN,0.0,0.0,0.0,{'49W000000000054X'},"{'BEYONDCOAL': {'BEYOND-NL-2'}, 'ENTSOE': {'49..."
98,Didcot,Hard Coal,CCGT,PP,United Kingdom,1490.000000,0.550000,1970.0,1998.0,2013.0,51.622300,-1.260800,NaN,0.0,0.0,0.0,{'48WSTN0000DIDCBC'},"{'BEYONDCOAL': {'BEYOND-UK-22'}, 'ENTSOE': {'4..."
129,Mellach,Hard Coal,Steam Turbine,CHP,Austria,200.000000,NaN,1986.0,1986.0,2020.0,46.911700,15.488300,NaN,0.0,0.0,0.0,{'14W-WML-KW-----0'},"{'BEYONDCOAL': {'BEYOND-AT-11'}, 'ENTSOE': {'1..."
150,Emile Huchet,Hard Coal,CCGT,PP,France,596.493211,NaN,1958.0,2010.0,2022.0,49.152500,6.698100,NaN,0.0,0.0,0.0,"{'17W100P100P0344D', '17W100P100P0345B'}","{'BEYONDCOAL': {'BEYOND-FR-67'}, 'ENTSOE': {'1..."
151,Amercoeur,Hard Coal,CCGT,PP,Belgium,451.000000,0.187765,1968.0,NaN,2009.0,50.431000,4.395500,NaN,0.0,0.0,0.0,{'22WAMERCO000010Y'},"{'BEYONDCOAL': {'BEYOND-BE-27'}, 'ENTSOE': {'2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29779,St,Hard Coal,NaN,CHP,Germany,21.645000,NaN,1982.0,NaN,NaN,49.976593,9.068953,NaN,0.0,0.0,0.0,{nan},{'MASTR': {'MASTR-SEE971943692655'}}
29804,Uer,Hard Coal,NaN,CHP,Germany,15.200000,NaN,1964.0,NaN,NaN,51.368132,6.662350,NaN,0.0,0.0,0.0,{nan},{'MASTR': {'MASTR-SEE988421065542'}}
29813,Walheim,Hard Coal,NaN,PP,Germany,244.000000,NaN,1964.0,NaN,NaN,49.017585,9.157690,NaN,0.0,0.0,0.0,"{nan, nan}","{'MASTR': {'MASTR-SEE937157344278', 'MASTR-SEE..."


**Task 3:** Identify the three largest coal power plants. In which countries are they located? When were they built?

In [28]:
coal.loc[coal.Capacity.nlargest(3).index]

,Name,Fueltype,Technology,Set,Country,Capacity,Efficiency,DateIn,DateRetrofit,DateOut,lat,lon,Duration,Volume_Mm3,DamHeight_m,StorageCapacity_MWh,EIC,projectID
id,,,,,,,,,,,,,,,,,,
194,Kozienice,Hard Coal,Steam Turbine,PP,Poland,3682.216205,NaN,1972.0,NaN,2042.0,51.6647,21.4667,NaN,0.0,0.0,0.0,"{'19W000000000104I', '19W000000000095U'}","{'BEYONDCOAL': {'BEYOND-PL-96'}, 'ENTSOE': {'1..."
3652,Vuglegirska,Hard Coal,CCGT,PP,Ukraine,3600.000000,NaN,1972.0,NaN,NaN,48.4652,38.2027,NaN,0.0,0.0,0.0,{nan},"{'GPD': {'WRI1005107'}, 'GEO': {'GEO-43001'}}"
767,Opole,Hard Coal,Steam Turbine,PP,Poland,3071.893939,NaN,1993.0,NaN,2020.0,50.7518,17.8820,NaN,0.0,0.0,0.0,{'19W0000000001292'},"{'BEYONDCOAL': {'BEYOND-PL-16'}, 'ENTSOE': {'1..."


**Task 4:** What is the average "DateIn" of each "Fueltype"? Which type of power plants is the oldest on average?

In [29]:
2024 - df.groupby("Fueltype").DateIn.mean().sort_values()

Fueltype
Hard Coal        52.092593
Hydro            51.232471
Nuclear          48.214953
Lignite          47.184211
Other            31.543103
Waste            26.845588
Geothermal       23.857143
Oil              23.410138
Solid Biomass    22.516588
Natural Gas      22.081505
Wind             14.483789
Biogas           11.415730
Solar             8.584491
Name: DateIn, dtype: float64

**Task 5:** In the time series provided, calculate the annual average capacity factors of wind and solar.

In [30]:
ts.mean()

load       54.736992
onwind      0.205556
offwind     0.362993
solar       0.122621
prices     31.835717
dtype: float64

**Task 6:** In the time series provided, calculate and plot the monthly average electricity price.

In [31]:
ts["prices"].resample("ME").mean().plot()